In [3]:
import netCDF4 as nc
import numpy as np
#ちゃんと反映されましたね
# 年代のリスト
years = [900, 1400, 1700, 1850, 1950, 2000]

for year in years:
    # netCDFファイルを開く(降水量を取得)
    if year==900:
        dataset1 = nc.Dataset('pr_Amon_GISS-E2-R_past1000_r1i1p121_085001-089912_modified.nc', 'r')
        variable1 = dataset1.variables['pr']
        # 変数をNumPyの配列に変換
        data_array1 = np.array(variable1)
        data_around_year1=data_array1[12*(year-29-850):, :, :]

        dataset2=nc.Dataset('pr_Amon_GISS-E2-R_past1000_r1i1p121_090001-094912_modified.nc', 'r')
        variable2 = dataset2.variables['pr']
        data_array2 = np.array(variable2)
        data_around_year2=data_array2[0:12, :, :]
        data_around_year=np.concatenate([data_around_year1, data_around_year2], axis=0)

        



    elif year==1400:
        dataset = nc.Dataset('pr_Amon_GISS-E2-R_past1000_r1i1p121_135101-140012_modified.nc', 'r')
        variable = dataset.variables['pr']# 変数をNumPyの配列に変換
        data_array = np.array(variable)
        data_around_year = data_array[12*(year-29-1351):, :, :]


    elif year==1700:
        dataset = nc.Dataset('pr_Amon_GISS-E2-R_past1000_r1i1p121_165101-170012_modified.nc', 'r')
        variable = dataset.variables['pr']# 変数をNumPyの配列に変換
        data_array = np.array(variable)
        data_around_year = data_array[12*(year-29-1651):, :, :]


    elif year==1850:
        dataset = nc.Dataset('pr_Amon_GISS-E2-R_past1000_r1i1p121_180101-185012_modified.nc', 'r')
        variable = dataset.variables['pr']# 変数をNumPyの配列に変換
        data_array = np.array(variable)
        data_around_year = data_array[12*(year-29-1801):, :, :]


    elif year==1950:
        dataset1 = nc.Dataset('pr_Amon_GISS-E2-R_historical_r1i1p121_190101-192512_modified.nc', 'r')
        variable1 = dataset1.variables['pr']
        # 変数をNumPyの配列に変換
        data_array1 = np.array(variable1)
        data_around_year1=data_array1[12*(year-29-1901):, :, :]

        dataset2=nc.Dataset('pr_Amon_GISS-E2-R_historical_r1i1p121_192601-195012_modified.nc', 'r')
        variable2 = dataset2.variables['pr']
        data_array2 = np.array(variable2)
        data_around_year2=data_array2[0:, :, :]
        data_around_year=np.concatenate([data_around_year1, data_around_year2], axis=0)

        

    else:
        dataset1 = nc.Dataset('pr_Amon_GISS-E2-R_historical_r1i1p121_195101-197512_modified.nc', 'r')
        variable1 = dataset1.variables['pr']
        # 変数をNumPyの配列に変換
        data_array1 = np.array(variable1)
        data_around_year1=data_array1[12*(year-29-1951):, :, :]

        dataset2=nc.Dataset('pr_Amon_GISS-E2-R_historical_r1i1p121_197601-200012_modified.nc', 'r')
        variable2 = dataset2.variables['pr']
        data_array2 = np.array(variable2)
        data_around_year2=data_array2[0:, :, :]
        data_around_year=np.concatenate([data_around_year1, data_around_year2], axis=0)

        
    
    # 年ごとの平均を計算
    annual_means = np.mean(data_around_year.reshape(-1, 12, data_around_year.shape[1], data_around_year.shape[2]), axis=0)
    

    # 単位変換のために全ての値に3600×24×365/12をかける
    conversion_factor = 3600 * 24 * 365 / 12
    annual_means *= conversion_factor

    
    # Create a new netCDF file for saving the annual mean data
    output_filename = f'annual_mean_pr_GISS121_{year}.nc'
    output_dataset = nc.Dataset(output_filename, 'w', format='NETCDF4')
    
    # Create dimensions in the new netCDF file based on the original dataset
    datasetA=nc.Dataset('pr_Amon_GISS-E2-R_past1000_r1i1p121_135101-140012_modified.nc', 'r')
    for dim_name, dim_size in datasetA.dimensions.items():
        if dim_name == 'time':
            output_dataset.createDimension(dim_name, 12)  # 12 months
        else:
            output_dataset.createDimension(dim_name, len(dim_size) if not dim_size.isunlimited() else None)
    
    # 緯度(lat)と経度(lon)のデータを取得
    lat_data = datasetA.variables['lat'][:]
    lon_data = datasetA.variables['lon'][:]

    # 新しいファイルに緯度(lat)と経度(lon)のディメンションを作成（既に存在しない場合のみ）
    if 'lat' not in output_dataset.dimensions:
        output_dataset.createDimension('lat', len(lat_data))
    if 'lon' not in output_dataset.dimensions:
        output_dataset.createDimension('lon', len(lon_data))

    # 緯度(lat)と経度(lon)の変数を作成（既に存在しない場合のみ）
    if 'lat' not in output_dataset.variables:
        lat_var = output_dataset.createVariable('lat', 'f4', ('lat',))
        lat_var[:] = lat_data  # 緯度データを割り当て
    if 'lon' not in output_dataset.variables:
        lon_var = output_dataset.createVariable('lon', 'f4', ('lon',))
        lon_var[:] = lon_data  # 経度データを割り当て
    
    
    # Create a variable in the new netCDF file to store the annual mean data
    output_variable = output_dataset.createVariable('annual_mean_pr', 'f4', ('time', 'lat', 'lon'))
    
    # Assign the annual mean data to the new variable
    output_variable[:] = annual_means
    
    # Copy global attributes from the original dataset to the new dataset
    for attr_name in datasetA.ncattrs():
        output_dataset.setncattr(attr_name, datasetA.getncattr(attr_name))
    
    # Close the datasets
    datasetA.close()
    output_dataset.close()